# classify_dktc

predict and evaluation 해서 dktc 데이터를 dktc_exp, dktc_imp 로 결과까지 내어서 데이터 dktc_exp, dktc_imp를 csv 파일로 추출하는 것 까지 실행하기  
(threshold를 기준으로 일정 확률 사잇값을 ood: imp 레이블로 추출)

In [ ]:
!pip install keras-bert
!pip install keras-radam
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for keras-bert: filename=keras_bert-0.89.0-py3-none-any.whl size=33517 sha256=f1e9aaa75799c3e610b8c36d862b6804e54fd19e008b75b6ce2e2b23b2d99839
  Stored in directory: /root/.cache/pip/wheels/4e/26/24/14ecbc0166364db7f5500164b7d796263cf3cd10c57e892180
  Created wheel for keras-transformer: filename=keras_transformer-0.40.0-py3-none-any.whl size=12303 sha256=6ca41cd427a43c773754324b742c57032e27a9ad0d76471857006702e7b3d15b
  Stored in directory: /root/.cache/pip/wheels/5e/d6/d1/c588c3b2b112c8f1173934995836ab2f2de8323cce99fa998f
  Created wheel for keras-e

In [ ]:
# 라이브러리 import 및 주요 라이브러리 버전 확인 
import wandb
from wandb.keras import WandbCallback

import tensorflow as tf

import pandas as pd
import numpy as np  
import re
import pickle
from sklearn.model_selection import train_test_split

import keras as keras
from keras.models import load_model
from keras import backend as K
from keras import Input, Model
from keras import optimizers

import codecs
from tqdm import tqdm
import shutil

import json

import os
import warnings

warnings.filterwarnings(action='ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
# tf.logging.set_verbosity(tf.logging.ERROR)

from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps

from keras_radam import RAdam
from keras_radam.training import RAdamOptimizer


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

root_path = '/content/drive/MyDrive/Colab Notebooks/AIFFELTHON/extra_test_0420'

In [ ]:
pwd

'/content'

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/AIFFELTHON/extra_test_0420

/content/drive/MyDrive/Colab Notebooks/AIFFELTHON/extra_test_0420


In [ ]:
import encoding_korbert as enc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pretrained_path ="./bert" #상대경로 잡기

SEQ_LEN = enc.SEQ_LEN

config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')

def build_model():
    model = load_trained_model_from_checkpoint(config_path,
                                                checkpoint_path,
                                                training=True,
                                                trainable=True,
                                                seq_len=SEQ_LEN)
    
    inputs = model.inputs[:2]
    dense = model.layers[-3].output
    
    outputs = keras.layers.Dense(1, activation='sigmoid', 
                               kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),
                              name = 'real_output')(dense)

    bert_model = keras.models.Model(inputs, outputs)

    return bert_model

model_extra = build_model()

In [ ]:
model_extra.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input-Token (InputLayer)       [(None, 256)]        0           []                               
                                                                                                  
 Input-Segment (InputLayer)     [(None, 256)]        0           []                               
                                                                                                  
 Embedding-Token (TokenEmbeddin  [(None, 256, 768),  91812096    ['Input-Token[0][0]']            
 g)                              (119547, 768)]                                                   
                                                                                                  
 Embedding-Segment (Embedding)  (None, 256, 768)     1536        ['Input-Segment[0][0]']    

In [ ]:
model_extra.load_weights('./saved/model_extra_weights_0419.h5')

In [ ]:
dktc_df = pd.read_csv("./data/merged_dktc_train.csv", usecols=['class','conversation']) #dktc_train (dktc는 train만 레이블이 달려 있어서 train만 써서 확인해봄)

In [ ]:
dktc_x, dktc_y = enc.load_data(dktc_df)

100%|██████████| 5567/5567 [00:03<00:00, 1796.53it/s]


In [ ]:
tensor_dktc = tf.convert_to_tensor(np.array(dktc_x))

In [ ]:
probs = model_extra.predict(x=(tensor_dktc[0,:,:], tensor_dktc[1,:,:]), verbose=1)
#p는 확률값들의 배열 (0에 가까울수록 exp 1 에 가까울수록 not)

174/174 [==============================] - 109s 588ms/step


In [ ]:
np.set_printoptions(threshold=50) #넘파이 출력 제어 , threshold=np.inf : 생략없이 출력

In [ ]:
probs #값을 확인하고 threshold_min, threshold_max 정하기

array([[9.9994433e-01],
       [9.9994051e-01],
       [9.9981767e-01],
       ...,
       [9.9994206e-01],
       [9.3151796e-01],
       [1.1923806e-04]], dtype=float32)

In [ ]:
probs.shape 

(5567, 1)

### dktc 데이터에서는 'not'이 없다.  
### 그래서 1에 가까울수록 not 이다.  
### 그러면 일정 값 이상의 확률을 가지는 데이터는 imp로 볼 수 있다.

In [ ]:
# threshold_min = 0.1
threshold_max = 0.995

indices = []
i=0
for p in probs:
    if threshold_max<p: 
        # print(p)
        indices.append(i)
    i+=1
            
    #(0: exp, 1: not)

In [ ]:
i_ =0
j=0
k=0
l=0
for p in probs:
  if p>=0 and p<=0.5:
    i_ += 1
  elif p>0.5 and p<=0.75:
    j += 1
  elif p>0.75 and p<=0.9:
    k +=1
  else:
    l +=1

print(len(probs),i_,j,k,l)

5567 292 122 160 4993


In [ ]:
indices

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 46,
 47,
 48,
 49,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 61,
 62,
 64,
 65,
 66,
 67,
 69,
 70,
 73,
 74,
 75,
 76,
 77,
 78,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 113,
 114,
 115,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 125,
 126,
 127,
 128,
 129,
 132,
 133,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 159,
 161,
 164,
 165,
 166,
 167,
 168,
 169,
 171,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 182,
 183,
 186,
 187,
 188,
 189,
 190,
 191,
 194,
 195,
 197,
 200,
 201,
 202,
 203,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,

In [ ]:
np.set_printoptions(threshold=10)


In [ ]:
for i in indices:
    print(dktc_df['conversation'].iloc[i])
    print('====================================================')

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
아, 당신 집이에요.
지금 그 근처 지나가는데 혹시 시간 되시면 같이 한잔하실래요?
어?
왜?
아니요, 같은 팀이고 같이 친해야 일 능률도 오르고 호흡도 맞잖아요?
그냥 팀원들 간 화합을 위해서 서는 거예요.
어, 저것들은 더 직급이 낮은데 왜 불러요?
업무의 중심축은 우리 둘이잖아.
어, 나는 단둘이서는 좀.
업무와 상관없으면 그냥 평일에 만나자.
아니, 넌 항상 이러더라?
내가 얘기하려고 부르는 거 아니야?
평일에는 다들 있어서 속 얘기 못 하잖아?
내가 신경 쓰는데 왜 이렇게 예민해?
다른 사람을 돌보는 게 어떤 기분인지 전혀 모르는 것 같아
나는 대전 할머니를 만나러 가야 하는데, 교통비가 없어.
어?
네 앱으로 표를 사.
어?
나한테 돈 줄 거야?
아니, 돈 없어.
그럼 나는 그 돈을 어디서 내?
이봐, 너 부자잖아, 네 집 사거리에서 제일 큰 국밥집을 하니까 손님이 끊이지 않는 거야.
그게 너랑 무슨 상관이야?
미스 김, 주말에 뭐해?
평소에는 빨래하고 쉬어요.
남자친구는 안 만나요?
글쎄요.
없어요?
그럼 시간 맞춰서 술 한잔 할래요?
글쎄요.
빨래할 시간은 있는데 일할 시간은 없다.
대리님, 제가 뭘 하면 좋을 것 같아요?
생각해 봤어요?
네?
무슨 생각을 해요.
본인이 뭘 해야 할지 생각도 안 하고 물어요?
그 정도 머리도 없는데 무슨 회사에서 일하겠다고.
죄송해요.
저는 입사한 지 일주일밖에 안 됐어요.
그것 때문에 회사가 월급을 주는 줄 아세요?
동기들과 떠들 생각 말고 일 생각을 하세요.
네.
죄송합니다.
정신없이 서 있지 말고 자리로 가겠죠?
머리가 나빠서 이런 것까지 다 가르쳐야 하나?
아닙니다.
죄송합니다
강민아, 내일은 언니 생일이니까 돈 좀 빌려줘.
뭐?
얼마 전에도 언니 생일이라고 빌려갔잖아.
아, 이번엔 언니 생일이야.
아직 지난달에 빌린 돈도 못 갚았어.
주겠다고 빨리 보내줘.
정말?
적당히 해.
정말?
알았어, 마지막이야.
빨리 줄게.
질릴 만큼 자주 쓸게.


In [ ]:
dktc_imp_df = dktc_df.loc[indices]
dktc_exp_df = dktc_df.loc[~dktc_df.index.isin(indices)]

In [ ]:
dktc_imp_df.to_csv('./data/dktc_imp_995e-3.csv')
dktc_exp_df.to_csv('./data/dktc_exp_995e-3.csv')